# keras-yolo3 Object Detection

In [ ]:
%tensorflow_version 1.x

In [ ]:
!pip3 uninstall keras
!pip3 install keras==2.1.5

Uninstalling Keras-2.2.5:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/Keras-2.2.5.dist-info/*
    /usr/local/lib/python3.6/dist-packages/docs/*
    /usr/local/lib/python3.6/dist-packages/keras/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.6/dist-packages/docs/md_autogen.py
    /usr/local/lib/python3.6/dist-packages/docs/update_docs.py
Proceed (y/n)? y
  Successfully uninstalled Keras-2.2.5
     |████████████████████████████████| 337kB 12.1MB/s 


In [ ]:
import keras
keras.__version__

In [ ]:
!git clone https://github.com/qqwweee/keras-yolo3
%cd keras-yolo3/
!ls

Cloning into 'keras-yolo3'...
remote: Enumerating objects: 144, done.
remote: Total 144 (delta 0), reused 0 (delta 0), pack-reused 144
Receiving objects: 100% (144/144), 151.07 KiB | 282.00 KiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/keras-yolo3
coco_annotation.py  kmeans.py	train_bottleneck.py  yolo.py
convert.py	    LICENSE	train.py	     yolov3.cfg
darknet53.cfg	    model_data	voc_annotation.py    yolov3-tiny.cfg
font		    README.md	yolo3		     yolo_video.py


In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights
!python3 convert.py yolov3.cfg yolov3.weights model_data/yolo_weights.h5

--2020-03-05 18:25:34--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M   168KB/s    in 7m 25s  

2020-03-05 18:33:00 (544 KB/s) - ‘yolov3.weights’ saved [248007048/248007048]

Using TensorFlow backend.
Loading weights.
Weights Header:  0 2 0 [32013312]
Parsing Darknet config.
Creating Keras model.


Parsing section net_0
Parsing section convolutional_0
conv2d bn leaky (3, 3, 3, 32)




2020-03-05 18:33:12.808605: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2020-03-05 18:33:12.861503: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2728bc0 initialized for platform Host (this does not guarantee that XLA will be used). De

In [ ]:
# Install Kaggle API
!pip3 install -q kaggle
!pip3 install -q kaggle-cli

     |████████████████████████████████| 81kB 10.0MB/s 
     |████████████████████████████████| 5.3MB 37kB/s 
     |████████████████████████████████| 112kB 56.8MB/s 
     |████████████████████████████████| 112kB 75.4MB/s 
     |████████████████████████████████| 51kB 9.5MB/s 


In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "<username>" 
os.environ['KAGGLE_KEY'] = "<key>"

!kaggle datasets download -d tannergi/microcontroller-detection

 60% 5.00M/8.34M [00:00<00:00, 11.9MB/s]
100% 8.34M/8.34M [00:00<00:00, 18.8MB/s]


In [ ]:
!unzip microcontroller-detection.zip > /dev/null

In [ ]:
!mv 'Microcontroller Detection' microcontroller_detection

## Converting data to be compatible with Yolo

TODO: Add link to github

In [ ]:
import pandas as pd
import argparse
import os


def csv_to_yolo(input_filepath, output_filepath, image_folder):
	df = pd.read_csv(input_filepath)
	output_file = open(output_filepath, 'w')
	df['class_id'] = df['class'].map(lambda x: df['class'].unique().tolist().index(x))
	for image_name in df['filename'].unique():
		image_path = os.path.join(image_folder,image_name).replace("\\", "/")
		output_file.write(f'{image_path} ')
		for index, row in df[(df['filename']==image_name)].iterrows():
			output_file.write(f'{row["xmin"]},{row["ymin"]},{row["xmax"]},{row["ymax"]},{row["class_id"]} ')
		output_file.write('\n')

In [ ]:
csv_to_yolo('microcontroller_detection/train_labels.csv', 'train.txt', 'microcontroller_detection/train')

In [ ]:
!cat train.txt

microcontroller_detection/train/IMG_20181228_101826.jpg 317,265,556,342,0 
microcontroller_detection/train/IMG_20181228_101855.jpg 398,243,592,382,1 
microcontroller_detection/train/IMG_20181228_101903.jpg 290,239,514,387,2 
microcontroller_detection/train/IMG_20181228_101915.jpg 194,200,669,531,3 
microcontroller_detection/train/IMG_20181228_102013.jpg 383,336,591,444,0 
microcontroller_detection/train/IMG_20181228_102020.jpg 332,259,538,357,2 
microcontroller_detection/train/IMG_20181228_102033.jpg 240,307,419,411,2 
microcontroller_detection/train/IMG_20181228_102041.jpg 324,151,574,391,3 
microcontroller_detection/train/IMG_20181228_102048.jpg 457,198,553,287,1 
microcontroller_detection/train/IMG_20181228_102155.jpg 383,226,511,330,2 
microcontroller_detection/train/IMG_20181228_102201.jpg 193,190,632,463,3 
microcontroller_detection/train/IMG_20181228_102206.jpg 333,277,446,377,0 
microcontroller_detection/train/IMG_20181228_102210.jpg 369,297,516,384,1 
microcontroller_detection

## Create label file

In [ ]:
classes_file = open('model_data/microcontroller_classes.txt', 'w')
classes = pd.read_csv('microcontroller_detection/train_labels.csv')['class'].unique().tolist()
print(classes)
for c in classes:
  classes_file.write(c + '\n')
classes_file.close()

['Arduino_Nano', 'Heltec_ESP32_Lora', 'ESP8266', 'Raspberry_Pi_3']


In [ ]:
!ls model_data

coco_classes.txt	     tiny_yolo_anchors.txt  yolo_anchors.txt
microcontroller_classes.txt  voc_classes.txt	    yolo_weights.h5


In [ ]:
!cat model_data/microcontroller_classes.txt

Arduino_Nano
Heltec_ESP32_Lora
ESP8266
Raspberry_Pi_3


## Change train.py file

This step doesn't work well in Colab and is a lot easier when you work locally.

In [ ]:
train = open('train.py', 'r').read().replace("classes_path = 'model_data/voc_classes.txt'", "classes_path = 'model_data/microcontroller_classes.txt'").replace("batch_size = 32", "batch_size = 8")
open('train2.py', 'w').write(train)

8342

## Train model

In [ ]:
!python3 train2.py

Using TensorFlow backend.








2020-03-05 18:37:05.149330: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2020-03-05 18:37:05.149569: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1d90bc0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-03-05 18:37:05.149605: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-03-05 18:37:05.151669: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-03-05 18:37:05.275690: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-03-05 18:37:05.276172: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1d90d80 initialized for platform CUDA (this does not guarantee that XLA will be u

In [ ]:
!ls model_data

coco_classes.txt	     tiny_yolo_anchors.txt  yolo_anchors.txt
microcontroller_classes.txt  voc_classes.txt	    yolo_weights.h5
